<a href="https://colab.research.google.com/github/azzindani/03_Prompt_Engineering/blob/main/PE_EN_SmolLM2_1_7B_Instruct_4bit_Med_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Modules

In [1]:
!pip install -U -q bitsandbytes
!pip install -q optimum
!pip install -q auto-gptq

In [2]:
import torch

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  GenerationConfig,
  BitsAndBytesConfig,
  GPTQConfig
)

if torch.cuda.is_available():
  print('GPU is available!')
else:
  print('GPU is not available.')

GPU is available!


## Import Model

In [3]:
model_name = 'HuggingFaceTB/SmolLM2-1.7B-Instruct'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit = True,
  bnb_4bit_quant_type = 'nf4',
  bnb_4bit_compute_dtype = torch.float16,
  bnb_4bit_use_double_quant = True,
)

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config = bnb_config,
  trust_remote_code = True
).to(device) #'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [6]:
'''model = AutoModelForCausalLM.from_pretrained(
  model_name,
  torch_dtype = torch.float16,
  trust_remote_code = True
).to(device) #'''

'model = AutoModelForCausalLM.from_pretrained(\n  model_name,\n  torch_dtype = torch.float16,\n  trust_remote_code = True\n).to(device) #'

## Import Tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

## Model Preview

In [8]:
def assistant(prompt):
  prompt_format = f"### Human: {prompt}\n### Assistant :"

  inputs = tokenizer(
    prompt_format,
    return_tensors = 'pt'
  ).to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    **inputs,
    generation_config = generation_config
  )
  return print(tokenizer.decode(outputs[0], skip_special_tokens = True))

In [9]:
# Level 1: Basic Prompt
prompt = "What are the symptoms and treatments of diabetes?"
assistant(prompt)

### Human: What are the symptoms and treatments of diabetes?
### Assistant :
Diabetes is a chronic condition that affects how your body regulates blood sugar levels. There are two main types of diabetes: Type 1 and Type 2.

**Type 1 Diabetes:**
Symptoms:
1. Frequent urination
2. Increased thirst and hunger
3. Fatigue
4. Blurred vision
5. Slow healing of cuts and wounds
6. Weight loss despite eating more

Treatments:
1. Insulin injections
2. Monitoring blood sugar levels
3. Balanced diet and exercise

**Type 2 Diabetes:**
Symptoms:
1. Frequent urination
2. Increased thirst and hunger
3. Fatigue
4. Blurred vision
5. Slow healing of cuts and wounds
6. Weight loss despite eating more

Treatments:
1. Lifestyle changes (diet, exercise, weight loss)
2. Oral medications (e.g., metformin)
3. Insulin injections (if necessary)

**Prevention:**
1. Regular exercise
2. Healthy diet
3. Maintaining a healthy weight
4. Regular check-ups with a healthcare provider

Remember, it's important to consult wi

In [10]:
# Level 2: Adding Detail
prompt = "Briefly describe the symptoms and treatments of diabetes."
assistant(prompt)

### Human: Briefly describe the symptoms and treatments of diabetes.
### Assistant :
Diabetes is a chronic condition that affects how your body regulates blood sugar levels. There are two main types of diabetes: Type 1 and Type 2.

Type 1 diabetes is an autoimmune disease where the body's immune system attacks and destroys the cells in the pancreas that produce insulin, a hormone that regulates blood sugar levels. Symptoms of Type 1 diabetes include increased thirst and urination, fatigue, weight loss, blurred vision, and slow healing of cuts and wounds.

Type 2 diabetes is a metabolic disorder where the body either doesn't produce enough insulin or doesn't use insulin effectively. Symptoms of Type 2 diabetes include increased thirst and urination, fatigue, weight loss, blurred vision, slow healing of cuts and wounds, and frequent infections.

Treatments for diabetes include lifestyle changes such as maintaining a healthy diet, regular exercise, and weight management. In some cases, me

In [11]:
# Level 3: Specify Information Type and Audience
prompt = "Explain the common symptoms and treatments of diabetes to a general audience."
assistant(prompt)

### Human: Explain the common symptoms and treatments of diabetes to a general audience.
### Assistant :
Diabetes is a chronic condition that affects how your body regulates blood sugar levels. There are two main types of diabetes: Type 1 and Type 2.

**Type 1 Diabetes:**
Type 1 diabetes is an autoimmune disease where the body's immune system attacks and destroys the cells in the pancreas that produce insulin, a hormone that regulates blood sugar levels. Symptoms include increased thirst and urination, fatigue, weight loss, and blurred vision.

**Type 2 Diabetes:**
Type 2 diabetes is a metabolic disorder where the body becomes resistant to insulin, making it harder for glucose to enter cells. Symptoms include increased thirst and urination, fatigue, weight loss, blurred vision, and slow healing of cuts and wounds.

**Treatments:**
Both types of diabetes require lifestyle changes, such as a healthy diet, regular exercise, and maintaining a healthy weight. For Type 1 diabetes, insulin in

In [12]:
# Level 4: Request for Structured Response
prompt = "List the main symptoms of diabetes, then outline the standard treatments."
assistant(prompt)

### Human: List the main symptoms of diabetes, then outline the standard treatments.
### Assistant :
1. **Human:** The main symptoms of diabetes are:
   - Increased thirst and hunger
   - Frequent urination
   - Fatigue
   - Slow-healing sores
   - Blurred vision
   - Erectile dysfunction
   - Weight loss
   - Erectile dysfunction
   - Slow-healing sores
   - Blurred vision
   - Increased thirst and hunger

2. **Standard Treatment:**
   - **Diet and Exercise:** A healthy diet and regular exercise can help manage diabetes.
   - **Medication:** Insulin or oral medications may be prescribed to control blood sugar levels.
   - **Regular Check-ups:** Regular monitoring and follow-up appointments with healthcare providers are crucial.
   - **Lifestyle Changes:** Smoking cessation, stress management, and adequate sleep are also important.
   - **Blood Sugar Monitoring:** Regular blood sugar testing is necessary to adjust treatment plans.
   - **Complications:** Early detection and treatment o

In [13]:
# Level 5: Differentiate by Type
prompt = "List the symptoms and treatments for both Type 1 and Type 2 diabetes."
assistant(prompt)

### Human: List the symptoms and treatments for both Type 1 and Type 2 diabetes.
### Assistant :
Here are the symptoms and treatments for both Type 1 and Type 2 diabetes:

**Type 1 Diabetes:**

* Symptoms:
	+ Frequent urination
	+ Increased thirst and hunger
	+ Fatigue
	+ Weight loss
	+ Blurred vision
* Treatments:
	+ Insulin therapy (injections or pump)
	+ Lifestyle changes (diet, exercise, stress management)
	+ Regular monitoring of blood sugar levels

**Type 2 Diabetes:**

* Symptoms:
	+ Frequent urination
	+ Increased thirst and hunger
	+ Fatigue
	+ Weight loss
	+ Blurred vision
* Treatments:
	+ Lifestyle changes (diet, exercise, stress management)
	+ Oral medications (e.g., metformin, sulfonylureas)
	+ Injectable medications (e.g., prandin, repaglinide)
	+ Regular monitoring of blood sugar levels

Note: This is a general overview and individual cases may vary. Always consult with a healthcare professional for accurate information and treatment plans.


In [14]:
# Level 6: Target a Professional Audience
prompt = "Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know."
assistant(prompt)

### Human: Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know.
### Assistant :
1. **Type 1 Diabetes:**
- **Symptoms:** Rapid weight loss, increased thirst and urination, fatigue, blurred vision, and frequent infections.
- **Diagnostic Criteria:** Fasting Plasma Glucose (FPG) or Oral Glucose Tolerance Test (OGTT) to confirm elevated blood glucose levels.
- **Treatment Options:** Insulin therapy, lifestyle modifications (diet, exercise), and regular monitoring.

### Type 2 Diabetes:
- **Symptoms:** Similar to Type 1, but often milder.
- **Diagnostic Criteria:** FPG or OGTT, along with a family history of diabetes and risk factors like obesity, physical inactivity, and smoking.
- **Treatment Options:** Lifestyle modifications (diet, exercise, weight loss), oral medications (metformin, sulfonylureas, DPP-4 inhibitors), and in some cases, insulin therapy.

2. **Key Points for Healthcare Provider

In [15]:
# Level 7: Clinical Detail with Disease Stages
prompt = "Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages."
assistant(prompt)

### Human: Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages.
### Assistant :
Diabetes is a chronic condition that affects how the body regulates blood sugar levels. There are three main types of diabetes: Type 1, Type 2, and prediabetes.

**Type 1 Diabetes:**
Type 1 diabetes is an autoimmune disease where the body's immune system attacks and destroys the insulin-producing beta cells in the pancreas. This results in little to no insulin production. Symptoms include:
- Frequent urination
- Increased thirst and hunger
- Fatigue
- Weight loss
- Blurred vision
- Slow healing of cuts and wounds
- Recurrent skin, gum, or bladder infections

**Type 2 Diabetes:**
Type 2 diabetes is the most common form of diabetes, accounting for 90-95% of all cases. It is characterized by insulin resistance, where the body's cells do not respond to insulin or do not produ

In [16]:
# Level 8: Add Nuance with Treatment Personalization
prompt = "Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors."
assistant(prompt)

### Human: Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors.
### Assistant :
Diabetes is a chronic condition that affects how the body regulates blood sugar levels. There are two main types of diabetes: Type 1 and Type 2.

**Type 1 Diabetes:**
Type 1 diabetes is an autoimmune disease where the body's immune system attacks and destroys the insulin-producing beta cells in the pancreas. This results in a complete lack of insulin production. Symptoms of Type 1 diabetes include:
- Frequent urination
- Increased thirst and hunger
- Fatigue
- Blurred vision
- Slow healing of cuts and wounds
- Weight loss despite increased appetite

Diagnosis involves blood tests to check for high blood sugar levels and confirm the absence of insulin production. Treatment involves insulin injections, lifestyle modifications, and regular monitoring 

In [17]:
# Level 9: Consider Risks, Complications, and Treatment Impacts
prompt = "Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment."
assistant(prompt)

### Human: Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment.
### Assistant :
Diabetes is a chronic condition that affects how the body regulates blood sugar levels. There are two main types of diabetes: Type 1 and Type 2.

**Type 1 Diabetes**

Type 1 diabetes, formerly known as juvenile diabetes, is an autoimmune disease where the body's immune system attacks and destroys the cells in the pancreas that produce insulin. This results in a complete lack of insulin production. Symptoms of Type 1 diabetes include increased thirst and urination, frequent urination, blurred vision, slow healing of cuts and wounds, and fatigue.

Diagnosis involves a blood test to check for the presence of insulin antibodies

In [18]:
# Level 10: Advanced Detail for Clinical Research or Specialist Training
prompt = "Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes."
assistant(prompt)

### Human: Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes.
### Assistant :
Diabetes is a chronic metabolic disorder characterized by elevated blood glucose levels due to insulin resistance or pancreatic beta-cell dysfunction. The two main types of diabetes are Type 1 and Type 2, with distinct clinical presentations, risk factors, and treatment approaches.

**Type 1 Diabetes**

Type 1 diabetes is an autoimmune disease where the immune system attacks and destroys pancreatic beta-cells, leading to insulin deficiency. Symptoms include:
- Frequent urination
- Increased t